# T-SNE 시각화

In [ ]:

from sklearn.manifold import TSNE
import torch
from torch.utils.data import DataLoader
import tqdm
import numpy as np
import time
import os
import matplotlib.pyplot as plt

from utils.visualize import draw_joint, denormalization
from cfg import classes
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from model.vid_cls import ResNet_TSM, Resnet_LSTM

# model = Custom_LSTM(image_features=2048,
#                     hidden_size=1024,
#                     num_layers=4,
#                     dropout=0, #Dropout을 하기 위해서는 레이어의 수가 2 이상이어야함
#                     bidirectional=True)

num_frames = 8
model = ResNet_TSM(num_layers=[3,4,23,3], frame=num_frames, heatmap=True)
model_save_path = f'./experiments/VC/{model.name}'

model.load_state_dict(torch.load(f'{model_save_path}/Last9.pth')['model'])
model = model.to(device)
model = model.eval()

In [ ]:
from dataset.get_data import get_dataset,split_data
dataset = get_dataset("./data/")
_, valid, _ = split_data(dataset,valid_ratio=0.2,valid_selection=0) # Split된 데이터 중 0번째를 Validation으로 사용한 경우

In [ ]:
from dataset.dataset import Video
batch_size = 16

tsne_valid_set = Video(valid, num_frames = num_frames, views = [1, 2, 3, 4, 5], training =  False, frame_mode=2)
tsne_valid_loader = DataLoader(tsne_valid_set,batch_size=batch_size,shuffle=False, num_workers=16)

In [ ]:
features = []
cls_probs = []
true_labels = []

progress = tqdm.tqdm(tsne_valid_loader,desc=f'Pred')
with torch.no_grad() :
    for i,data in enumerate(progress):
        output = model(data['video_heatmap'].to(device))

        feature = output['feature'].cpu().detach().numpy()

        cls_prob = output['class'].cpu().detach().numpy()

        b,c = cls_prob.shape

        if (model.name == "TSM"):
            feature = np.mean(feature.reshape(b,num_frames,-1),1)

        for j in range(b):
            features.append(feature[j])
            cls_probs.append(cls_prob[j])
            true_labels.append(data['class'][j].numpy())

features = np.array(features)
cls_probs = np.array(cls_probs)
true_labels = np.array(true_labels)

np.save(f'{model_save_path}/features',features)
np.save(f'{model_save_path}/cls_probs',cls_probs)
np.save(f'{model_save_path}/true_labels',true_labels)

In [ ]:
features = np.load(f'{model_save_path}/features.npy')
cls_probs = np.load(f'{model_save_path}/cls_probs.npy')
true_labels = np.load(f'{model_save_path}/true_labels.npy')
cls = np.argmax(cls_probs,-1)


## 전체 클래스 시각화

In [ ]:
# features는 T-SNE를 적용하고자 하는 특정 모델의 feature 벡터입니다.
random_state = 42

if os.path.exists(f'{model_save_path}/tsne_{random_state}.npy'):
    tsne_result = np.load(f'{model_save_path}/tsne_{random_state}.npy')
else:
    start_time = time.time()
    tsne = TSNE(n_components=2, random_state=random_state)
    tsne_result = tsne.fit_transform(features)
    print((time.time()-start_time)/60,"분")
    np.save(f'{model_save_path}/tsne_{random_state}',tsne_result)

In [ ]:
# 각 클래스에 대한 고유한 색상을 정의합니다.
unique_labels = np.unique(cls)
colors = plt.cm.tab20(np.linspace(0, 1, len(unique_labels)))

# 각 클래스를 반복하면서 해당 클래스의 feature를 플롯합니다.
for i, label in enumerate(unique_labels):
    class_indices = np.where(cls == label)[0]
    plt.scatter(tsne_result[class_indices, 0], tsne_result[class_indices, 1], label=f'Class {label}', color=colors[i], s = 5)

# 플롯을 설정하고 레이블을 추가합니다.
plt.xlabel('T-SNE Dimension 1')
plt.ylabel('T-SNE Dimension 2')
plt.legend(loc='best')
plt.title(f'{model_save_path.split("/")[-1]} T-SNE Visualization')
plt.show()


## 시점별 TSNE 시각화


In [ ]:
target_idx = [2,3,5]
views = [0,1,2,3,4]
idx_list = []
for cls_idx in range(len(classes)):
    for view in views:
        for idx in range(len(valid[cls_idx][view])):
            for frame in range(max(1,len(valid[cls_idx][view][idx][3])-num_frames+1)):
                idx_list.append([cls_idx,view,idx,frame])

idx_list = np.array(idx_list)
idx_list.shape

(113530, 4)

In [ ]:
# features는 T-SNE를 적용하고자 하는 특정 모델의 feature 벡터입니다.
random_state = 42

if os.path.exists(f'{model_save_path}/tsne_{random_state}.npy'):
    tsne_result = np.load(f'{model_save_path}/tsne_{random_state}.npy')
else:
    start_time = time.time()
    tsne = TSNE(n_components=2, random_state=random_state)
    tsne_result = tsne.fit_transform(features)
    print((time.time()-start_time)/60,"분")
    np.save(f'{model_save_path}/tsne_{random_state}',tsne_result)

In [ ]:

for i in range(5):
    target_bool = (idx_list[:,1] == i)
    target_result = tsne_result[target_bool]
    #target_label = cls[target_bool]
    target_label = true_labels[target_bool]

    unique_labels = np.unique(target_label)
    colors = plt.cm.tab20(np.linspace(0, 1, len(unique_labels)))

    # 각 클래스를 반복하면서 해당 클래스의 feature를 플롯합니다.
    for j, label in enumerate(unique_labels):
        class_indices = np.where(target_label == label)[0]
        plt.scatter(target_result[class_indices, 0], target_result[class_indices, 1], label=f'Class {label}', color=colors[j], s = 5)

    # 플롯을 설정하고 레이블을 추가합니다.
    plt.xlabel('T-SNE Dimension 1')
    plt.ylabel('T-SNE Dimension 2')
    plt.legend(loc='best')
    plt.title(f'{model_save_path.split("/")[-1]} T-SNE Visualization( View {i+1} )')
    plt.show()


## 세개 클래스 T- SNE 시각화

In [ ]:
random_state = 42
target_idx = [2,3,5]
target_bool = np.zeros(features.shape[0])
for t in target_idx:
    target_bool = target_bool + (idx_list[:,0] == t)
target_bool = (target_bool >= 1)


target_feature = features[target_bool]
#target_label = cls[target_bool]
target_label = true_labels[target_bool]
if os.path.exists(f'{model_save_path}/tsne_{random_state}_3cls.npy'):
    tsne_result = np.load(f'{model_save_path}/tsne_{random_state}_3cls.npy')
else:
    tsne = TSNE(n_components=2, random_state=random_state)
    tsne_result = tsne.fit_transform(target_feature)
    np.save(f'{model_save_path}/tsne_{random_state}_3cls',tsne_result)

In [ ]:
# 각 클래스에 대한 고유한 색상을 정의합니다.
unique_labels = np.unique(true_labels)
colors = plt.cm.tab20(np.linspace(0, 1, len(unique_labels)))

# 각 클래스를 반복하면서 해당 클래스의 feature를 플롯합니다.
for i, label in enumerate(unique_labels):
    class_indices = np.where(target_label == label)[0]
    plt.scatter(tsne_result[class_indices, 0], tsne_result[class_indices, 1], label=f'Class {label}', color=colors[i], s = 5)

# 플롯을 설정하고 레이블을 추가합니다.
plt.xlabel('T-SNE Dimension 1')
plt.ylabel('T-SNE Dimension 2')
plt.legend(loc='best')
plt.title(f'{model_save_path.split("/")[-1]} T-SNE Visualization ( 3 Cls only )')
plt.show()


In [ ]:

img = tsne_valid_set[43168]['video_heatmap'][:,:3]
heatmap = tsne_valid_set[43168]['video_heatmap'][:,3:]
coord = tsne_valid_set[43168]['coord']

img = np.transpose(img,(0,2,3,1))
print(img.shape)

plt.figure(figsize=(5*num_frames,10))
for j in range(num_frames):
    plt.subplot(1,num_frames,j+1)
    plt.imshow(draw_joint(((denormalization(img[j]))*255).numpy().astype(np.uint8()),torch.tensor(coord[j])))
    plt.title(f"{data['class']}({j})")
plt.show()


### 시점별 시각화

In [ ]:
idx_list[target_bool][:,1]==1

In [ ]:

for i in range(5):
    next_target_bool = (idx_list[target_bool][:,1] == i)
    target_result = tsne_result[next_target_bool]
    target_label = cls[target_bool][next_target_bool]

    unique_labels = np.unique(true_labels)
    colors = plt.cm.tab20(np.linspace(0, 1, len(unique_labels)))

    # 각 클래스를 반복하면서 해당 클래스의 feature를 플롯합니다.
    for j, label in enumerate(unique_labels):
        class_indices = np.where(target_label == label)[0]
        plt.scatter(target_result[class_indices, 0], target_result[class_indices, 1], label=f'Class {label}', color=colors[j], s = 5)

    # 플롯을 설정하고 레이블을 추가합니다.
    plt.xlabel('T-SNE Dimension 1')
    plt.ylabel('T-SNE Dimension 2')
    plt.legend(loc='best')
    plt.title(f'{model_save_path.split("/")[-1]} T-SNE Visualization( View {i+1} )')
    plt.show()
